In [2]:
import pandas as pd
import numpy as np
import json
from urllib.request import urlopen
from pandas.io.json import json_normalize

In [3]:
def refactor_values(data):
    def transform_value(value):
        if isinstance(value, str) and '<' in value:
            return int(value.split('<')[1])
        else:
            return value

    return {key: transform_value(value) for key, value in data.items() if value != ''}

In [4]:
file = open('../data/raw.json', 'r', encoding='utf-8')
raw = json.load(file)
file.close()

df = pd.DataFrame()

for i in range(len(raw)):
    jdata = json.loads(raw[i]['json'])
    station = jdata['arsopodatki']['postaja']
    for i in range(len(station)):
        data = station[i]
        data = refactor_values(data)
        df = pd.concat([df, pd.json_normalize(data)])  

In [5]:
df

,nadm_visina,datum_od,merilno_mesto,benzen,ge_sirina,sifra,datum_do,ge_dolzina,pm2.5,o3,pm10,co,no2,so2
0,299,2023-02-20 21:00,LJ Bežigrad,1.0,46.065851,E403,2023-02-20 22:00,14.517454,NaN,NaN,NaN,NaN,NaN,NaN
0,305,2023-02-20 21:00,LJ Celovška,NaN,46.072399,E405,2023-02-20 22:00,14.491849,NaN,NaN,NaN,NaN,NaN,NaN
0,293,2023-02-20 21:00,LJ Vič,NaN,46.037791,E404,2023-02-20 22:00,14.494001,NaN,NaN,NaN,NaN,NaN,NaN
0,388,2023-02-20 21:00,Kranj,NaN,46.242115,E417,2023-02-20 22:00,14.366963,NaN,NaN,NaN,NaN,NaN,NaN
0,270,2023-02-20 21:00,MB Titova,1.5,46.559202,E407,2023-02-20 22:00,15.656191,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,241,2023-02-19 02:00,Zagorje,NaN,46.131416,E415,2023-02-19 03:00,15.000906,32.0,84.0,34.0,NaN,8.0,3.0
0,290,2023-02-19 02:00,Hrastnik,NaN,46.144213,E414,2023-02-19 03:00,15.088097,28.0,NaN,29.0,NaN,NaN,NaN
0,220,2023-02-19 02:00,Novo mesto,NaN,45.802080,E418,2023-02-19 03:00,15.182060,33.0,72.0,35.0,NaN,6.0,NaN
0,540,2023-02-19 02:00,Iskrba,0.0,45.561344,E420,2023-02-19 03:00,14.862816,12.0,88.0,14.0,NaN,NaN,1.0


In [6]:
print(df.isnull().sum())

nadm_visina          0
datum_od             0
merilno_mesto        0
benzen           71808
ge_sirina         3685
sifra                0
datum_do             0
ge_dolzina        3685
pm2.5            11270
o3               41650
pm10             11270
co               74057
no2              43608
so2              67916
dtype: int64


In [7]:
cat_cols = df.select_dtypes(include=['object']).columns
print(cat_cols)
num_cols = df.select_dtypes(exclude=['object']).columns
print(num_cols)

Index(['datum_od', 'merilno_mesto', 'sifra', 'datum_do'], dtype='object')
Index(['nadm_visina', 'benzen', 'ge_sirina', 'ge_dolzina', 'pm2.5', 'o3',
       'pm10', 'co', 'no2', 'so2'],
      dtype='object')


In [8]:
df = df.assign(**{col: df[col].fillna(df[col].mean()) for col in num_cols})
df = pd.concat([df, pd.get_dummies(df['merilno_mesto'])], axis=1)
df.drop(columns=['merilno_mesto', 'sifra', 'datum_od', 'datum_do'], inplace=True)

df

,nadm_visina,benzen,ge_sirina,ge_dolzina,pm2.5,o3,pm10,co,no2,so2,...,MB Vrbanski,MS Cankarjeva,MS Rakičan,NG Grčna,Novo mesto,Otlica,Ptuj,Rečica v I.Bistrici,Trbovlje,Zagorje
0,299,1.000000,46.065851,14.517454,31.185601,54.150301,37.192392,0.411268,21.755218,2.176576,...,0,0,0,0,0,0,0,0,0,0
0,305,1.040595,46.072399,14.491849,31.185601,54.150301,37.192392,0.411268,21.755218,2.176576,...,0,0,0,0,0,0,0,0,0,0
0,293,1.040595,46.037791,14.494001,31.185601,54.150301,37.192392,0.411268,21.755218,2.176576,...,0,0,0,0,0,0,0,0,0,0
0,388,1.040595,46.242115,14.366963,31.185601,54.150301,37.192392,0.411268,21.755218,2.176576,...,0,0,0,0,0,0,0,0,0,0
0,270,1.500000,46.559202,15.656191,31.185601,54.150301,37.192392,0.411268,21.755218,2.176576,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,241,1.040595,46.131416,15.000906,32.000000,84.000000,34.000000,0.411268,8.000000,3.000000,...,0,0,0,0,0,0,0,0,0,1
0,290,1.040595,46.144213,15.088097,28.000000,54.150301,29.000000,0.411268,21.755218,2.176576,...,0,0,0,0,0,0,0,0,0,0
0,220,1.040595,45.802080,15.182060,33.000000,72.000000,35.000000,0.411268,6.000000,2.176576,...,0,0,0,0,1,0,0,0,0,0
0,540,0.000000,45.561344,14.862816,12.000000,88.000000,14.000000,0.411268,21.755218,1.000000,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df_sorted = df.sort_index(axis=1)

In [10]:
df_sorted

,CE Ljubljanska,CE bolnica,Hrastnik,Iskrba,Koper,Kranj,Krvavec,LJ Bežigrad,LJ Celovška,LJ Vič,...,benzen,co,ge_dolzina,ge_sirina,nadm_visina,no2,o3,pm10,pm2.5,so2
0,0,0,0,0,0,0,0,1,0,0,...,1.000000,0.411268,14.517454,46.065851,299,21.755218,54.150301,37.192392,31.185601,2.176576
0,0,0,0,0,0,0,0,0,1,0,...,1.040595,0.411268,14.491849,46.072399,305,21.755218,54.150301,37.192392,31.185601,2.176576
0,0,0,0,0,0,0,0,0,0,1,...,1.040595,0.411268,14.494001,46.037791,293,21.755218,54.150301,37.192392,31.185601,2.176576
0,0,0,0,0,0,1,0,0,0,0,...,1.040595,0.411268,14.366963,46.242115,388,21.755218,54.150301,37.192392,31.185601,2.176576
0,0,0,0,0,0,0,0,0,0,0,...,1.500000,0.411268,15.656191,46.559202,270,21.755218,54.150301,37.192392,31.185601,2.176576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0,0,0,0,0,0,0,0,0,0,...,1.040595,0.411268,15.000906,46.131416,241,8.000000,84.000000,34.000000,32.000000,3.000000
0,0,0,1,0,0,0,0,0,0,0,...,1.040595,0.411268,15.088097,46.144213,290,21.755218,54.150301,29.000000,28.000000,2.176576
0,0,0,0,0,0,0,0,0,0,0,...,1.040595,0.411268,15.182060,45.802080,220,6.000000,72.000000,35.000000,33.000000,2.176576
0,0,0,0,1,0,0,0,0,0,0,...,0.000000,0.411268,14.862816,45.561344,540,21.755218,88.000000,14.000000,12.000000,1.000000


In [13]:
df = df_sorted
df.to_csv('../data/processed.csv', index=False)